# SQL via Pandas

# How to use df.to_sql() function

In [1]:
car_names.to_sql(name = 'car_names', con = connection, if_exists = 'replace', index = False)

NameError: name 'car_names' is not defined

In [2]:
car_makers_csv = '../datasets/csv/car-makers.csv' #same like before
makers = pd.read_csv(car_makers_csv, encoding = 'utf-8') #same like before
makers.to_sql(name = 'car_makers', con = connection, if_exists = 'replace', index = False)

NameError: name 'pd' is not defined

In [ ]:
car_data_csv = '../datasets/csv/cars-data.csv'
data = pd.read_csv(car_data_csv, encoding = 'utf-8')
data.to_sql(name = 'car_data', con = connection, if_exists = 'replace', index = False)

In [ ]:
ferrari = (None, 'Ferrari','The Ferrari')
tesla = [None, 'Tesla', None]

In [ ]:
CARS.execute('INSERT INTO car_names VALUES (?, ?, ?)', ferrari)
CARS.execute('INSERT INTO car_names VALUES (?, ?, ?)',tesla)

# Connect via pandas to sqlite3

In [9]:
# Noel try full code without going everywhere.
import pandas as pd
from pandas.io import sql
import sqlite3
# pandas.read_sql()
conn = sqlite3.connect('./datasets/sql/Cars.db.sqlite')
results = sql.read_sql('select * from car_names', conn)

In [10]:
left_join = sql.read_sql('SELECT car_names.*, car_data.*'
                          'FROM car_names '
                          'LEFT JOIN car_data '    # also 'FULL OUTER JOIN', 'RIGHT JOIN', 'LEFT JOIN', INNER JOIN
                          'ON car_names."Id"=car_data."Id"', conn)

left_join.head()

,Id,Model,Make,Id,MPG,Cylinders,Edispl,Horsepower,Weight,Accelerate,Year
0,1,'chevrolet','chevrolet chevelle malibu',1,18.0,8,307.0,130.0,3504,12.0,1970
1,2,'buick','buick skylark 320',2,15.0,8,350.0,165.0,3693,11.5,1970
2,3,'plymouth','plymouth satellite',3,18.0,8,318.0,150.0,3436,11.0,1970
3,4,'amc','amc rebel sst',4,16.0,8,304.0,150.0,3433,12.0,1970
4,5,'ford','ford torino',5,17.0,8,302.0,140.0,3449,10.5,1970


# psycopg2 is PostgreSQL

In [12]:
from sqlalchemy import create_engine
import psycopg2
import pandas as pd
# Noel Question to Edo, what is the differnce to connect via sqlite3.connect() vs psycopg2.connect()
conn_str = "host='dsi.c20gkj5cvu3l.us-east-1.rds.amazonaws.com' dbname='northwind' user='dsi_student' password='gastudents'"
conn = psycopg2.connect(conn_str)

In [13]:
# ORDER BY
# Noel, How to check column names? This info is needed to code the following. I have tried wildcard, didn't work.
df = pd.read_sql(
'SELECT "ProductID" as "PID","ProductName","SupplierID","UnitPrice"'
'FROM Products '
'WHERE Products."UnitPrice" > 25 '
'ORDER BY "SupplierID" DESC,"UnitPrice" ASC', con=conn)   #DESC descending order
df.head()

,PID,ProductName,SupplierID,UnitPrice
0,61,Sirop d'érable,29,28.5
1,62,Tarte au sucre,29,49.3
2,60,Camembert Pierrot,28,34.0
3,59,Raclette Courdavault,28,55.0
4,56,Gnocchi di nonna Alice,26,38.0


In [14]:
# Trying XX AS XX => Aliasing
SQL_STRING = '''
SELECT "SupplierID" AS "Supplier No.", "CompanyName" AS "Company Name"  
FROM Suppliers AS S
ORDER BY "CompanyName" ASC
'''
df = pd.read_sql(SQL_STRING, con=conn)
df.head()

,Supplier No.,Company Name
0,18,Aux joyeux ecclésiastiques
1,16,Bigfoot Breweries
2,5,Cooperativa de Quesos 'Las Cabras'
3,27,Escargots Nouveaux
4,1,Exotic Liquids


In [15]:
# LIKE  => WHERE XXX LIKE  XXX like searching for a pattern  
# % is wildcard any string that contain string 'ch'.
# if S% => means 
# Noel pls note, LIKE quotation must be 'XXX' , cannot  "XXX".
# Noel: Seems case sensitive. How to search none case sensitive?
SQL_STRING = '''
SELECT "ProductName" AS "Ch Products"
FROM Products
WHERE "ProductName" LIKE '%ch%'
ORDER BY "ProductName" DESC
'''
df = pd.read_sql(SQL_STRING, con=conn)
df

,Ch Products
0,Schoggi Schokolade
1,Sasquatch Ale
2,Queso Manchego La Pastora
3,Pâté chinois
4,Gumbär Gummibärchen
5,Gnocchi di nonna Alice


In [16]:
# Try DISTINCT is used to return only distinct (unique) values.
# Noel: DOes this means DISTINCT applies to all 3x columns? What if I want DISTINCT only one column, but display others as non-distinct?

SQL_STRING = '''
SELECT DISTINCT "SupplierID", "ProductName", "UnitPrice"
FROM Products as P
ORDER BY P."UnitPrice" ASC
'''
df = pd.read_sql(SQL_STRING, con=conn)
df.head()

,SupplierID,ProductName,UnitPrice
0,15,Geitost,2.50
1,10,Guaraná Fantástica,4.50
2,6,Konbu,6.00
3,24,Filo Mix,7.00
4,25,Tourtière,7.45


In [17]:
# Try LIMIT: Sometimes, we may want to only retrieve a fixed number of records from a database.
SQL_STRING = '''
SELECT "ProductName" AS "Top 5 A Products", "UnitPrice"
FROM products
WHERE "ProductName" LIKE '%a%'
ORDER BY "UnitPrice" DESC  
LIMIT 5
'''
df = pd.read_sql(SQL_STRING, con=conn)
df

,Top 5 A Products,UnitPrice
0,Côte de Blaye,263.50
1,Thüringer Rostbratwurst,123.79
2,Sir Rodney's Marmalade,81.00
3,Carnarvon Tigers,62.50
4,Raclette Courdavault,55.00


In [18]:
# Try SUB
SQL_STRING = '''
SELECT SUB."Top 5 A Products", sub."UnitPrice"
FROM 
        (SELECT "ProductName" AS "Top 5 A Products", "UnitPrice"
        FROM products
        WHERE "ProductName" LIKE '%a%'
        ORDER BY "UnitPrice" DESC  
        LIMIT 5) AS SUB
ORDER BY "UnitPrice" ASC
'''
df = pd.read_sql(SQL_STRING, con=conn)
df

,Top 5 A Products,UnitPrice
0,Raclette Courdavault,55.00
1,Carnarvon Tigers,62.50
2,Sir Rodney's Marmalade,81.00
3,Thüringer Rostbratwurst,123.79
4,Côte de Blaye,263.50


In [19]:
# Try GROUPBY
# Noel: Why is the count and sum not written in WHERE?
SQL_STRING = '''
SELECT  "OrderID"
        ,COUNT("OrderID") AS "Noel_count"
        ,SUM("UnitPrice"*"Quantity") AS "Noel_revenue"

FROM order_details
GROUP BY "OrderID"
ORDER BY 3 DESC;
'''
df = pd.read_sql(SQL_STRING, con=conn)
df.head()

,OrderID,Noel_count,Noel_revenue
0,10865,2,17250.000000
1,11030,4,16321.900082
2,10981,1,15810.000000
3,10372,4,12281.200180
4,10424,3,11493.200127


In [20]:
#Show the revenue of all orders with more than one item.

In [21]:
SQL_STRING = '''
SELECT  "OrderID"
        ,COUNT("OrderID") AS OrderCount
        ,SUM("UnitPrice"*"Quantity") AS "revenue"
FROM order_details
GROUP BY "OrderID"
HAVING COUNT("OrderID") > 1
ORDER BY "revenue" DESC;
'''
df = pd.read_sql(SQL_STRING, con=conn)
df.head()

,OrderID,ordercount,revenue
0,10865,2,17250.000000
1,11030,4,16321.900082
2,10372,4,12281.200180
3,10424,3,11493.200127
4,10817,4,11490.699940


In [22]:
SQL_STRING = '''
SELECT "CompanyName"
     ,"City"
     ,"Country"
     ,CASE 
        WHEN "Country" = 'USA' THEN 'domestic'
        ELSE 'foreign'
     END AS "NOEL"
FROM Suppliers
'''
pd.read_sql(SQL_STRING, con=conn).head()

,CompanyName,City,Country,NOEL
0,Exotic Liquids,London,UK,foreign
1,New Orleans Cajun Delights,New Orleans,USA,domestic
2,Grandma Kelly's Homestead,Ann Arbor,USA,domestic
3,Tokyo Traders,Tokyo,Japan,foreign
4,Cooperativa de Quesos 'Las Cabras',Oviedo,Spain,foreign


In [23]:
SQL_STRING = '''
SELECT "OrderDate"
       ,EXTRACT("year" from "OrderDate") AS year_float
       ,CAST(EXTRACT("year" from "OrderDate") AS Int) AS year_int
FROM Orders
'''
# Total freight per month
df = pd.read_sql(SQL_STRING, con=conn)
df.head()

,OrderDate,year_float,year_int
0,1996-07-04,1996.0,1996
1,1996-07-05,1996.0,1996
2,1996-07-08,1996.0,1996
3,1996-07-08,1996.0,1996
4,1996-07-09,1996.0,1996


In [24]:
# more example on CAST, EXTRACT, HAVING
# Nested Query
SQL_STRING = '''
SELECT "year", "month", SUM("Freight") as "FreightPerMonth"
FROM (SELECT "OrderDate",
    CAST(EXTRACT("year" from "OrderDate") AS Int) AS year,
    CAST(EXTRACT('month' from "OrderDate") AS Int) AS month,
    CAST(EXTRACT(day FROM "OrderDate") AS Int) AS day,
    "Freight"
FROM Orders) as sub
GROUP BY year, month
HAVING SUM("Freight") > 5000
ORDER BY year DESC, month DESC

'''
# Total freight per month
df = pd.read_sql(SQL_STRING, con=conn)
df

,year,month,FreightPerMonth
0,1998,4,6393.57
1,1998,3,5379.02
2,1998,1,5463.44


In [25]:
# Average  SQL column
SQL_STRING = '''
SELECT "ShipCountry",
       AVG("ShippedDate" - "OrderDate") as "avg_shipping_time"
FROM orders
WHERE "ShippedDate" IS NOT NULL
GROUP BY "ShipCountry"
ORDER BY "avg_shipping_time" DESC
'''
df = pd.read_sql(SQL_STRING, con=conn)
df.head()

,ShipCountry,avg_shipping_time
0,Ireland,11.000000
1,Sweden,10.216216
2,Switzerland,9.941176
3,USA,9.554622
4,Argentina,9.285714


In [46]:
# Noel not sure how to do, but understood the codes here.
SQL_STRING = '''

SELECT "ShipCountry", AVG("Freight") "AvgFreight"
FROM orders
WHERE CAST(EXTRACT(year FROM "OrderDate") AS Int) = 1998
GROUP BY "ShipCountry"
ORDER BY "AvgFreight" DESC
LIMIT 5
'''

df = pd.read_sql(SQL_STRING, con=conn)
df.head()

,ShipCountry,AvgFreight
0,Ireland,339.422489
1,Austria,217.341821
2,USA,153.322308
3,Portugal,102.470003
4,Germany,90.987941


In [48]:
# ALTERNATIVE
query = """
           SELECT "ShipCountry", AVG("Freight") "AvgFreight"
           FROM Orders
           WHERE "ShippedDate">='1998-01-01' AND 
                 "ShippedDate"<'1998-12-31'           
           GROUP BY "ShipCountry"
           ORDER BY AVG("Freight") DESC
           
        """
pd.read_sql(query, con=conn).head()

,ShipCountry,AvgFreight
0,Ireland,339.422489
1,Austria,199.137275
2,USA,166.906216
3,Denmark,131.617501
4,Portugal,102.470003


In [ ]:
# Nested query or SUB query
SQL_STRING = '''

SELECT "Gender", "HireDate"
FROM (SELECT "TitleOfCourtesy", "HireDate",
            CASE WHEN "TitleOfCourtesy" IN ('Mrs.', 'Ms.') 
                THEN 'Female'
                ELSE 'Male' 
            END AS "Gender"
        FROM employees) as genders
WHERE genders."Gender" = 'Female'
ORDER BY "HireDate" DESC
limit 2
'''

df = pd.read_sql(SQL_STRING, con=conn)
df.head()

In [73]:
# Nested query or SUB query
# Comments inside SELECT
SQL_STRING = '''

SELECT "Price_Indicator", COUNT("Price_Indicator"), MIN("UnitPrice"), MAX("UnitPrice"), AVG("UnitPrice")
FROM (
      -- NOTE: the columns that are not used are filtered out!
      SELECT "UnitPrice",
              CASE WHEN "UnitPrice" > 50 THEN 'Expensive'
                   WHEN "UnitPrice" < 10 THEN 'Cheap'
                   ELSE 'Fair' 
              END AS "Price_Indicator"
      FROM Products) AS New_Table
Group By New_Table."Price_Indicator"

'''

df = pd.read_sql(SQL_STRING, con=conn)
df.head()

,Price_Indicator,count,min,max,avg
0,Cheap,11,2.5,9.65,7.459091
1,Fair,59,10.0,49.30,23.768983
2,Expensive,7,53.0,263.50,105.112857
